In [ ]:
conda install -c conda-forge auto-sklearn

In [ ]:
pip install -r requirements.txt

In [4]:
import pandas as pd
import autosklearn.regression

from feature_engine import transformation as vt

ModuleNotFoundError: No module named 'autosklearn'

In [ ]:
df_train = pd.read_csv("../data/train.csv")
df_test = pd.read_csv("../data/test.csv")

In [12]:
transformer = vt.YeoJohnsonTransformer(variables=["Lenght", "Diameter", "Height", "Whole Weight", "Shucked Weight", "Viscra Weight", "Shell Weight"])
transformer.fit(df_train)
df_train = transformer.transform(df_train)

In [13]:
df_train = pd.concat([df_train.drop(columns=["Gender"]), pd.get_dummies(df_train["Gender"])], axis=1)

In [14]:
X = df_train.drop(columns=["id", "Target"]).to_numpy()
y = df_train["Target"].to_numpy()

In [18]:
automl = autosklearn.regression.AutoSklearnRegressor(n_jobs=-1, resampling_strategy="cv",
                                                    resampling_strategy_arguments={"cv": 10, "folds": 10}, metric=autosklearn.metrics.mean_squared_error)
automl.fit(X, y)                                                    

[WARNING] [2022-03-23 12:27:06,137:Client-AutoML(1):1c59bac4-aa59-11ec-9695-2cf05d84cf26] Time limit for a single run is higher than total time limit. Capping the limit for a single run to the total time given to SMAC (3599.744692)
[WARNING] [2022-03-23 12:27:06,137:Client-AutoML(1):1c59bac4-aa59-11ec-9695-2cf05d84cf26] Capping the per_run_time_limit to 1799.0 to have time for a least 2 models in each process.


AutoSklearnRegressor(metric=mean_squared_error, n_jobs=-1,
                     per_run_time_limit=8640, resampling_strategy='cv',
                     resampling_strategy_arguments={'cv': 10, 'folds': 10})

In [16]:
df_test["Target"] = 0
df_test = transformer.transform(df_test)
df_test = pd.concat([df_test.drop(columns=["Gender"]), pd.get_dummies(df_test["Gender"])], axis=1)
X_test = df_test.drop(columns=["id", "Target"])
y_preds = automl.predict(X_test)

In [17]:
df_submission = pd.read_csv("../data/sample_submission.csv")
df_submission["Target"] = y_preds
df_submission.to_csv("../results/V4.csv", index=False)